<a href="https://colab.research.google.com/github/lfzuloaga-dot/Proyecto-Margen-Costo/blob/main/Gisela_Costo_Ingreso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Producto**

In [2]:
import pandas as pd
import numpy as np
import re
import warnings

# Suprimir todas las advertencias de openpyxl
warnings.simplefilter(action='ignore', category=UserWarning)

# Ruta del archivo. Asegúrate de que este nombre coincida exactamente con tu Excel.
archivo = "DETALLE INGRESOS y COSTOS LLC 2025.xlsx"

# Abrir el archivo de Excel para acceder a todas las hojas
try:
    xls = pd.ExcelFile(archivo)
    hojas = xls.sheet_names
except FileNotFoundError:
    print(f"Error: El archivo '{archivo}' no se encontró. Asegúrate de que está en la misma carpeta que el script.")
    input("Presiona Enter para salir...")
    exit()

# Diccionario para mapear abreviaturas de meses a nombres completos
meses_map = {
    'ene': 'Enero', 'feb': 'Febrero', 'mar': 'Marzo', 'abr': 'Abril',
    'may': 'Mayo', 'Jun': 'Junio', 'jul': 'Julio', 'agto': 'Agosto',
    'sept': 'Septiembre', 'oct': 'Octubre', 'nov': 'Noviembre', 'dic': 'Diciembre'
}

# Diccionario para almacenar los totales por mes
resumen_mensual = {}

for nombre_hoja in hojas:
    nombre_hoja_lower = nombre_hoja.lower()

    # Identificar el mes de la hoja de manera robusta
    mes_encontrado = None
    for abreviacion, nombre_mes in meses_map.items():
        if abreviacion in nombre_hoja_lower:
            mes_encontrado = nombre_mes
            break

    if not mes_encontrado:
        continue

    df = pd.read_excel(xls, sheet_name=nombre_hoja)

    if 'Importe en moneda' not in df.columns:
        continue

    df['Importe en moneda'] = pd.to_numeric(df['Importe en moneda'], errors='coerce')
    suma_hoja = df['Importe en moneda'].sum()

    if mes_encontrado not in resumen_mensual:
        resumen_mensual[mes_encontrado] = {'Ingresos': 0.0, 'Costos': 0.0}

    if '401010' in nombre_hoja:
        resumen_mensual[mes_encontrado]['Ingresos'] += suma_hoja
    elif '501010' in nombre_hoja:
        resumen_mensual[mes_encontrado]['Costos'] += suma_hoja

# Convertir el diccionario a un DataFrame de pandas para un mejor formato
df_resumen = pd.DataFrame.from_dict(resumen_mensual, orient='index')
df_resumen.index.name = 'Mes'
df_resumen.reset_index(inplace=True)

# Calcular el margen y el margen porcentual para cada mes
df_resumen['Margen'] = df_resumen['Ingresos'] - df_resumen['Costos']

# Cálculo de la fórmula de margen de ganancia sobre el costo
df_resumen['Margen (%)'] = np.where(
    df_resumen['Costos'] != 0,
    (abs(df_resumen['Ingresos']) - df_resumen['Costos']) / df_resumen['Costos'] * 100,
    np.nan
)

# Formato contable latinoamericano
def formato_monto(valor):
    if pd.isna(valor) or valor == 0:
        return "0,00"
    return f"{valor:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")

# Formatear las columnas numéricas
df_resumen['Ingresos'] = df_resumen['Ingresos'].apply(formato_monto)
df_resumen['Costos'] = df_resumen['Costos'].apply(formato_monto)
df_resumen['Margen (%)'] = df_resumen['Margen (%)'].apply(lambda x: f"{int(round(x))} %" if pd.notna(x) else "0%")

# Imprimir la tabla, excluyendo la columna 'Margen'
print("\n--- Resumen de Ganancias y Pérdidas por Mes ---")
print(df_resumen[['Mes', 'Ingresos', 'Costos', 'Margen (%)']].to_string(index=False))

# Mantener la ventana abierta hasta que el usuario presione Enter
input("\nPresiona Enter para cerrar la ventana...")


--- Resumen de Ganancias y Pérdidas por Mes ---
       Mes    Ingresos     Costos Margen (%)
     Enero        0,00  71.202,86     -100 %
   Febrero -187.463,71 151.267,79       24 %
     Marzo -205.690,18 142.617,63       44 %
     Abril -210.885,99 163.255,82       29 %
      Mayo -803.568,09 625.809,52       28 %
     Julio -188.263,84 146.771,20       28 %
    Agosto -218.199,30 175.676,83       24 %
Septiembre -215.428,56 144.152,61       49 %
   Octubre -268.406,50 182.360,40       47 %

Presiona Enter para cerrar la ventana...


''

# **JSON**

In [4]:

{
  "income_codes": ["401010"],
  "cost_codes":   ["501010"],

  "importe_candidates": [
    "Importe en moneda",
    "Importe",
    "Monto",
    "Importe (moneda)",
    "Importe en Moneda"
  ],

  "month_tokens": {
    "Enero":       ["ene", "enero"],
    "Febrero":     ["feb", "febrero"],
    "Marzo":       ["mar", "marzo"],
    "Abril":       ["abr", "abril"],
    "Mayo":        ["may", "mayo"],
    "Junio":       ["jun", "junio"],
    "Julio":       ["jul", "julio"],
    "Agosto":      ["ago", "agto", "agosto"],
    "Septiembre":  ["sep", "sept", "septiembre"],
    "Octubre":     ["oct", "octubre"],
    "Noviembre":   ["nov", "noviembre"],
    "Diciembre":   ["dic", "diciembre"]
  },

  "margin_basis": "costs",

  "output": {
    "csv_path": "outputs/resumen_mensual.csv",
    "json_path": "outputs/resumen_mensual.json",
    "pretty_json":
}


SyntaxError: '{' was never closed (ipython-input-3959758487.py, line 1)

# **Ruta Posible**

finresumen/
├─ README.md
├─ finresumen.py
├─ config.json                     # (mínimo) o config_avanzado.json
├─ data/
│  └─ DETALLE INGRESOS y COSTOS LLC 2025.xlsx   # ejemplo real
├─ outputs/
│  ├─ resumen_mensual.csv
│  └─ resumen_mensual.json
└─ tests/
   ├─ test_sample.xlsx
   └─ test_finresumen.py